Script to extract movie details from BBFC site and convert the same into Pandas dataframe

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

'''Function which scrapes recently rated page on BBFC site'''
def recent_page_film_scrape():
    '''
    Function scrapes recently rated page on BBFC site
    Iterate over each recently rated record and extracts href
    For every href, call scrape_film function and get dictionary with movie details
    Adds the extracted dictionary into a list and converts the list to a Pandas Dataframe

    Returns : Pandas dataframe containing movie details
    '''
    response_rr = requests.get(f"https://www.bbfc.co.uk/recently-rated")
    recently_rated_detail = []
    if response_rr.status_code == 200:
        recently_rated_html_content = response_rr.content
        recently_rated_soup = BeautifulSoup(recently_rated_html_content, 'html.parser')
        main_recent_added = recently_rated_soup.find('main', class_ = "Main_Main__46590")
        recent_added_list = main_recent_added.find_all('div', class_ = "SearchItem_SearchItem__3kbZF")
        for recent_added in recent_added_list:
            '''Detailed way to scrape information from movie page(DRY)'''
            anchor_tag = recent_added.find('a')
            href_rec = anchor_tag.get('href')
            recent_details_dict = scrape_film(href_rec)
            if type(recent_details_dict) == dict:
                recently_rated_detail.append(recent_details_dict)
            else:
                print(recent_details_dict)

            '''Alternate way to scrape information from recent rated page itself(Gives limited information)'''
            #recent_dict = {}
            #rating_tag = recent_added.find('span', class_ = "Icon_Icon__9RCS8 SearchItem_Rating__2fbS8")['aria-label'].split()[1]
            #recent_tag_title = anchor_tag.find('h3').text
            #category = anchor_tag.find('span').text
            #release_date = recent_added.find('div', class_ ="Type_base__2EnB2 SearchItem_ReleaseDate__21k53")
            # if release_date != None:
            #     release_date = release_date.text.split()[2]
            # else:
            #     release_date = 'N/A'
            # print(f"{recent_tag_title}, {category}")
            # print(href_rec)
            # recent_dict["recent_tag_title"] = recent_tag_title
            # recent_dict["release_date"] = release_date
            # recent_dict["rating_tag"] = rating_tag
            # recent_dict["href_rec"] = href_rec
            # recent_dict["category"] = category
            #recently_rated_detail.append(recent_details_dict)
        recent_added_film_df = pd.DataFrame(recently_rated_detail)
        json_data_recent = recent_added_film_df.to_json(orient='records')
        #print(json_data_recent)
        return recent_added_film_df
    else:
        return f'Recent added page not found with response status: {response_rr.status_code}'



def scrape_film(movie_href):
    '''
    Function scrapes the films page using href of specific film

    parameters:
    movie_href: string with href of a particular film e.g : /release/dada-q29sbgvjdglvbjpwwc0xmdexmty1

    return parameters:
    Success scenario : Dictionary of movie details
    Failure scenario : String containing error details

    '''
    response = requests.get(f"https://www.bbfc.co.uk{movie_href}")
    if response.status_code == 200:
        movie_details = {}
        movie_html_content = response.content
        movie_soup = BeautifulSoup(movie_html_content, 'html.parser')

        '''Extracting film title'''
        title_h2 = movie_soup.find('h2', class_ = "MediaOutline_Title__3UY5z")
        span = title_h2.find('span')
        title = span.text
        movie_details["title"]= title

        '''Extracting film rating'''
        rating_span = movie_soup.find('span', class_= "Icon_Icon__9RCS8 MediaOutline_Icon__3mXHF")
        movie_rating = rating_span['aria-label'].split()[1]
        movie_details["rating"] = movie_rating

        '''Extracting media type'''
        media_type_div = movie_soup.find('div', class_ ="MediaOutline_MediaType__AipEd")
        media_type = media_type_div.text
        movie_details["media_type"] = media_type

        '''Extracting key details'''
        keyList_ul = movie_soup.find('ul', class_ ="KeyDetails_List__1oCpD")
        key_li_list = keyList_ul.find_all('li', class_ ="ListItem_ListItem__klTTg")
        key_details = {}
        for li in key_li_list:
            h4_list = li.find_all('h4', class_ ="Type_base__2EnB2")
            if h4_list[0].text == 'Release date':
                release_date = datetime.datetime.strptime(h4_list[1].text, '%d/%m/%Y').date()
                key_details[h4_list[0].text] = str(release_date)
            else:
                key_details[h4_list[0].text] = h4_list[1].text
        movie_details["key_details"] = key_details

        '''Extracting content advice data'''
        content_details = movie_soup.find('div', class_ ="VisualAdvice_VisualAdvice__icKUz")
        if content_details != None:
            grid_items = content_details.find_all('div', class_ = "GridItem_GridItem__Wegeu")
            content_advice = []
            for grid_item in grid_items:
                advice_details = {}
                content_title = grid_item.find('p',class_= "Type_base__2EnB2").text
                max_rating_count = len(grid_item.find_all('span'))
                current_rating_count = max_rating_count - len(grid_item.find_all('span', class_ ="VisualAdviceRating_unfilled__1D3DY"))
                advice_details["content_title"] = content_title
                advice_details["content_rating"] = current_rating_count
                advice_details["max_rating"] = max_rating_count
                content_advice.append(advice_details)
            movie_details["content_advice"] = content_advice
        else:
            print(f"content details are not available for {movie_details['title']}")

        '''returning movie details dictionary with all the details'''
        return movie_details
    else:
        return f'Movie not found with response status: {response.status_code}'

def scrape_spreadsheet():
    '''
    Function scrapes the given excel sheet and returns the list of movies in the excel

    Return:
    list of movies in the sheet. e.g ['Creed III', 'Dada',...]
    '''
    response_spr = requests.get("https://docs.google.com/spreadsheets/d/1gdNU75_RPG69bsuAFQpZu670sHK9EzTwZWSLUERhMZ4/edit#gid=0")
    title_list = []
    if response_spr.status_code == 200:
      response_scr_cont = response_spr.content
      content_html_spr = BeautifulSoup(response_scr_cont, 'html.parser')
      xl_div = content_html_spr.find('div',id = '0-grid-table-container')
      tbody = xl_div.find('tbody')
      tr_spr_list = tbody.find_all('tr')
      for tr_spr in tr_spr_list:
          td = tr_spr.find('td')
          if td.text != '' and td.text != 'Title':
              title_list.append(td.text)
    else:
      print(f"Excel doc not found")
    return title_list

''''''
def scrape_query_result():
    '''
    Function iterates over a list of title extracted from scrape_spreadsheet() function
    Creates a query URL for each title and extracts query result page
    For every first query result extract the href and calls scrape_film function
    Adds the extracted dictionary into a list and converts the list to a Pandas Dataframe

    Returns : Pandas dataframe containing movie details


    '''
    title_list = scrape_spreadsheet()
    title_details_list = []
    for title in title_list:
        url_title =f"https://www.bbfc.co.uk/search?q={title}"
        response_title = requests.get(url_title)
        if response_title.status_code == 200:
            query_soup = BeautifulSoup(response_title.content, 'html.parser')
            result_div = query_soup.find('div', class_ ="GridItem_GridItem__Wegeu GridItem_m-100__32Nyd GridItem_t-100__Pq2Bq GridItem_tl-67__1OyMa")
            not_found = result_div.find('p',class_='BBFCSearchLayout_Title__1GdfR')
            if not_found == None:
                first_title = result_div.find('div', class_ ="SearchItem_SearchItem__3kbZF")
                anchor_tag_title = first_title.find('a')
                href_title = anchor_tag_title.get('href')
                title_details_dict = scrape_film(href_title)
                if type(title_details_dict) == dict:
                    title_details_list.append(title_details_dict)
                else:
                    print(title_details_dict)
            else:
                print(f"No results found for movie {title}")
        else:
            print(f"Query page returned with status code : {response_title.status_code} for {title}")
    title_details_df = pd.DataFrame(title_details_list)
    json_data = title_details_df.to_json(orient='records',date_format='iso')
    #print(json_data)
    return title_details_df


Use the below code to run the above functions

1. To extract a speoific movie details call the function as below:

In [ ]:
scrape_film("/release/dada-q29sbgvjdglvbjpwwc0xmdexmty1")

2. To extract movie details for each file in the recently Rated Page  call the functions as below:

In [ ]:
recent_page_film_scrape()

3. To extract the list of Titles from the spreadsheet , call the function as below:

In [ ]:
scrape_spreadsheet()

4. To extract the details for each title in the spreadsheet , call the function as below:

In [ ]:
scrape_query_result()

Results extracted in JSON format for both the dataframes.
